In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm 
from sklearn.metrics import classification_report,r2_score,accuracy_score,recall_score,precision_score,f1_score, confusion_matrix, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [ ]:
df = pd.read_csv("mhealth_raw_data.csv")
df

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
plt.figure(figsize=(10,8))
df['Activity'].value_counts().plot.bar()

In [ ]:
#since the data for 0 is too big, we will try to minimize it to match the other data
data_activity_0 = df[df["Activity"]==0]
data_activity_else = df[df["Activity"]!=0]
data_activity_0 = data_activity_0.sample(n=40000)
df=pd.concat([data_activity_0,data_activity_else])

In [ ]:
plt.figure(figsize=(10,8))
df['Activity'].value_counts().plot.bar()

In [ ]:
activity_label = {
    0: "none",
    1: "Standing still (1 min)",
    2: "Sitting and relaxing (1 min)",
    3: "Lying down (1 min)",
    4: "Walking (1 min)",
    5: "InClimbing stairs (1 min)",
    6: "Waist bends forward (20x)",
    7: "Frontal elevation of arms (20x)",
    8: "Knees bending (crouching) (20x)",
    9: "Cycling (1 min)",
    10: "Jogging (1 min)",
    11: "Running (1 min)",
    12: "Jump front & back (20x)"
}

In [ ]:
subject1 = df[df['subject'] == 'subject1']
readings = ['a', 'g']
for i in range(1, 13):
    for r in readings:
        print(f"=========================={activity_label[i]} - {r}==========================")
        plt.figure(figsize=(14, 4))
        plt.subplot(1, 2, 1)
        plt.plot(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'lx'],
                 color='blue', alpha=0.7, label=r + 'lx')
        plt.plot(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'ly'],
                 color='red', alpha=0.7, label=r + 'ly')
        plt.plot(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'lz'],
                 color='orange', alpha=0.7, label=r + 'lz')
        plt.title("Left ankle sensor")
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'rx'],
                 color='blue', alpha=0.7, label=r + 'rx')
        plt.plot(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'ry'],
                 color='red', alpha=0.7, label=r + 'ry')
        plt.plot(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'rz'],
                 color='orange', alpha=0.7, label=r + 'rz')
        plt.title("Right wrist sensor")
        plt.legend()
        plt.show()

In [ ]:
for i in range(1, 13):
    for r in readings:
        print(f"=========================={activity_label[i]} - {r}==========================")
        plt.figure(figsize=(14, 4))
        plt.subplot(1, 2, 1)
        plt.hist(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'lx'],
                 color='blue', alpha=0.7, label=r + 'lx')
        plt.hist(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'ly'],
                 color='red', alpha=0.7, label=r + 'ly')
        plt.hist(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'lz'],
                 color='orange', alpha=0.7, label=r + 'lz')
        plt.title("Left ankle sensor")
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.hist(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'rx'],
                 color='blue', alpha=0.7, label=r + 'rx')
        plt.hist(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'ry'],
                 color='red', alpha=0.7, label=r + 'ry')
        plt.hist(subject1[subject1['Activity'] == i].reset_index(drop=True)[r + 'rz'],
                 color='orange', alpha=0.7, label=r + 'rz')
        plt.title("Right wrist sensor")
        plt.legend()
        plt.show()

In [ ]:
df['Activity'] = df['Activity'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12],["none","Standing still (1 min)","Sitting and relaxing (1 min)","Lying down (1 min)","Walking (1 min)","InClimbing stairs (1 min)","Waist bends forward (20x)","Frontal elevation of arms (20x)","Knees bending (crouching) (20x)", "Cycling (1 min)","Jogging (1 min)","Running (1 min)","Jump front & back (20x)"])
df["Activity"]

In [ ]:
df1 = df.copy()
for feature in df1.columns[:-2]:
    lower_range = np.quantile(df[feature], 0.01)
    upper_range = np.quantile(df[feature], 0.99)
    print(feature,"range: ", lower_range, "to", upper_range)

    df1 = df1.drop(df1[(df1[feature] > upper_range) | (df1[feature] < lower_range)].index)

In [ ]:
#data processing
le = LabelEncoder()
df['subject'] = le.fit_transform(df['subject'])
df['Activity'] = le.fit_transform(df["Activity"])
df.plot(kind='box',subplots = True, layout = (5,5),figsize=(20,15))
plt.show()

In [ ]:
x = df.drop(['Activity','subject'],axis=1).values
y = df['Activity'].values 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)
ro_scaler = RobustScaler().fit(x_train)
x_train_scaled = ro_scaler.transform(x_train)
x_test_scaled = ro_scaler.transform(x_test)

In [ ]:
#model
def resultsSummerizer(y_true,y_pred , cm_en = True):
    cm = confusion_matrix(y_true,y_pred)
    acc = accuracy_score(y_true,y_pred)
    prec = precision_score(y_true,y_pred,average='macro')
    rec = sensitivity = recall_score(y_true,y_pred,average='macro')
    f1 = f1_score(y_true,y_pred,average= "macro")
    if cm_en:
        plt.figure(figsize=(15,15))
        sns.heatmap(cm, annot= True,cmap="Blues",xticklabels=activity_label.values(),yticklabels=activity_label.values())
        plt.title('confusion_matrix')
        plt.show()
    print(f"Accuracy Score: "+"{:.4%}".format(acc))
    print(f"Precision Score: "+"{:.4%}".format(prec))
    print(f"Recall Score: "+"{:.4%}".format(rec))
    print(f"F1_sc Score: "+"{:.4%}".format(f1))

In [ ]:
#logistic regression
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr.score(x_train, y_train)

In [ ]:
lr.score(x_test, y_test)
lr2 = LogisticRegression()
lr2.fit(x_train_scaled,y_train)
lr2.score(x_train_scaled,y_train)


In [ ]:
lr2.score(x_test_scaled,y_test)

In [ ]:
y_pred_lr = lr2.predict(x_test_scaled)

In [ ]:
# KNN
knn1 = KNeighborsClassifier(n_neighbors=5)
knn1.fit(x_train, y_train)


In [ ]:
y_pred_knn = knn1.predict(x_test)
resultsSummerizer(y_test,y_pred_knn,cm_en=False)

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors=5)
knn2.fit(x_train_scaled,y_train)
y_pred_knn = knn2.predict(x_test_scaled)
resultsSummerizer(y_test,y_pred_knn,cm_en=False)

In [ ]:
for n in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(x_train_scaled,y_train)
    y_pred = knn.predict(x_test_scaled)
    print(f"No of Neighbours: {n}\n")
    resultsSummerizer(y_test,y_pred,cm_en=False )


In [ ]:
dt = DecisionTreeClassifier(max_depth=1)
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_test)
resultsSummerizer(y_test,y_pred_dt)